In [1]:
%load_ext autoreload
%autoreload 2
import sys

# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

In [2]:
import pandas as pd
import datetime
from datetime import datetime
import time
import numpy as np
from datetime import timedelta
from datetime import date

In [7]:
df1 = pd.read_parquet("/home/aevans/ai2es/lstm/fh_02/2023/20230630_hrrr_fh02.parquet")

In [8]:
df1

,time,y,x,t2m,pt,sh2,d2m,r2,u10,v10,...,dswrf,dlwrf,tcc,gh,dpt,latitude,longitude,valid_time,atmosphere,isobaricInhPa


In [22]:
df2 = pd.read_parquet("/home/aevans/ai2es/lstm/fh_02/2022/20220912_hrrr_fh02.parquet")

In [23]:
df2

,time,y,x,t2m,pt,sh2,d2m,r2,u10,v10,...,dswrf,dlwrf,tcc,gh,dpt,latitude,longitude,valid_time,atmosphere,isobaricInhPa
205655,2022-09-12 00:00:00,1.383248e+06,2.447727e+06,288.764008,287.878571,0.01064,288.282166,92.300003,-1.404145,-0.682514,...,0.0,332.700012,3.0,5794.990723,236.624786,46.977215,-64.975092,2022-09-12 02:00:00,0.0,500.0
205654,2022-09-12 00:00:00,1.383248e+06,2.444724e+06,288.764008,287.816071,0.01058,288.219666,91.800003,-1.341645,-0.495014,...,0.0,333.299988,4.0,5795.178223,236.249786,46.986440,-65.011773,2022-09-12 02:00:00,0.0,500.0
205653,2022-09-12 00:00:00,1.383248e+06,2.441721e+06,288.889008,287.941071,0.01067,288.344666,92.099998,-1.279145,-0.370014,...,0.0,333.899994,6.0,5795.365723,235.999786,46.995655,-65.048464,2022-09-12 02:00:00,0.0,500.0
205652,2022-09-12 00:00:00,1.383248e+06,2.438718e+06,288.951508,288.003571,0.01073,288.407166,92.000000,-1.279145,-0.307514,...,0.0,334.600006,7.0,5795.553223,235.812286,47.004860,-65.085166,2022-09-12 02:00:00,0.0,500.0
205651,2022-09-12 00:00:00,1.383248e+06,2.435715e+06,289.139008,288.128571,0.01087,288.657166,92.699997,-1.216645,-0.370014,...,0.0,334.799988,6.0,5795.709473,235.687286,47.014054,-65.121879,2022-09-12 02:00:00,0.0,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-09-12 23:00:00,1.952773e+04,1.108106e+06,291.149658,292.311523,0.00758,282.787994,54.099998,0.516924,2.312595,...,0.0,318.500000,0.0,5709.304688,242.374786,37.973587,-84.837865,2022-09-13 01:00:00,0.0,500.0
3,2022-09-12 23:00:00,1.952773e+04,1.105102e+06,290.712158,291.999023,0.00782,283.225494,56.500000,0.579424,2.437595,...,0.0,318.000000,0.0,5709.210938,241.874786,37.977281,-84.871766,2022-09-13 01:00:00,0.0,500.0
2,2022-09-12 23:00:00,1.952773e+04,1.102098e+06,290.899658,292.186523,0.00764,282.850494,54.900002,0.516924,2.437595,...,0.0,318.500000,0.0,5709.148438,241.312286,37.980966,-84.905670,2022-09-13 01:00:00,0.0,500.0
1,2022-09-12 23:00:00,1.952773e+04,1.099094e+06,291.087158,292.436523,0.00762,282.850494,54.200001,0.391924,2.562595,...,0.0,319.299988,0.0,5709.085938,240.812286,37.984641,-84.939578,2022-09-13 01:00:00,0.0,500.0


In [8]:
the_df = pd.DataFrame()

In [76]:
# consider the start date as 2021-february 1 st
start_date = datetime(2017, 1, 1, 1, 0, 0)

# consider the end date as 2021-march 1 st
end_date = datetime(2017, 1, 5, 23, 59, 59)

# delta time
delta = timedelta(days=1)

In [77]:
savedir = "/home/aevans/ai2es/cleaned/HRRR/"
fh = "02"

In [78]:
# while start_date <= end_date:
#     print(start_date.strftime("%Y-%m-%d"))
#     start_date += delta

In [79]:
savedir = "/home/aevans/ai2es/cleaned/HRRR/"
# delta time
delta = timedelta(days=1)
while start_date <= end_date:
    the_df = pd.DataFrame()
    my_date = start_date
    my_time = my_date + timedelta(hours=int(fh))
    for i in np.arange(0, 24):
        my_time = str(my_time)
        print(my_time)
        init = str(i).zfill(2)
        month = str(my_date.strftime("%m"))
        year = str(my_date.strftime("%Y"))
        day = str(my_date.strftime("%d"))
        if (
            os.path.exists(
                f"{savedir}{year}/{month}/{year}{month}{day}_hrrr.t{init}z_{fh}.parquet"
            )
            == False
        ):
            continue
        else:
            df = pd.read_parquet(
                f"{savedir}{year}/{month}/{year}{month}{day}_hrrr.t{init}z_{fh}.parquet"
            ).reset_index()
            new_df = df[df["valid_time"] == my_time]
            the_df = pd.concat([new_df, the_df])

            time_obj = datetime.strptime(my_time, "%Y-%m-%d %H:%M:%S")
            my_time = time_obj + timedelta(hours=1)
    start_date += delta

    the_df.sort_values("valid_time", ascending=True)

2017-01-01 03:00:00
2017-01-01 04:00:00
2017-01-01 05:00:00
2017-01-01 06:00:00
2017-01-01 07:00:00
2017-01-01 08:00:00
2017-01-01 09:00:00
2017-01-01 10:00:00
2017-01-01 11:00:00
2017-01-01 11:00:00
2017-01-01 12:00:00
2017-01-01 13:00:00
2017-01-01 14:00:00
2017-01-01 15:00:00
2017-01-01 16:00:00
2017-01-01 17:00:00
2017-01-01 18:00:00
2017-01-01 18:00:00
2017-01-01 19:00:00
2017-01-01 20:00:00
2017-01-01 21:00:00
2017-01-01 22:00:00
2017-01-01 23:00:00
2017-01-02 00:00:00
2017-01-02 03:00:00
2017-01-02 04:00:00
2017-01-02 05:00:00
2017-01-02 06:00:00
2017-01-02 07:00:00
2017-01-02 08:00:00
2017-01-02 09:00:00
2017-01-02 10:00:00
2017-01-02 11:00:00
2017-01-02 12:00:00
2017-01-02 13:00:00
2017-01-02 14:00:00
2017-01-02 14:00:00
2017-01-02 14:00:00
2017-01-02 15:00:00
2017-01-02 16:00:00
2017-01-02 17:00:00
2017-01-02 18:00:00
2017-01-02 19:00:00
2017-01-02 20:00:00
2017-01-02 21:00:00
2017-01-02 22:00:00
2017-01-02 22:00:00
2017-01-02 23:00:00
2017-01-03 03:00:00
2017-01-03 04:00:00


In [81]:
the_df.iloc[::-1]

,time,y,x,t2m,pt,sh2,d2m,r2,u10,v10,...,dswrf,dlwrf,tcc,gh,dpt,latitude,longitude,valid_time,atmosphere,isobaricInhPa
205655,2017-01-05 02:00:00,1.383248e+06,2.447727e+06,272.661316,273.544342,0.00300,269.758972,81.000000,-0.386578,2.174433,...,0.0,311.700012,97.250,5312.041992,231.562286,46.977215,-64.975092,2017-01-05 04:00:00,0.0,500.0
205654,2017-01-05 02:00:00,1.383248e+06,2.444724e+06,272.473816,273.356842,0.00336,271.321472,90.000000,-0.011578,1.861933,...,0.0,310.299988,97.250,5310.541992,231.499786,46.986440,-65.011773,2017-01-05 04:00:00,0.0,500.0
205653,2017-01-05 02:00:00,1.383248e+06,2.441721e+06,273.036316,273.919342,0.00349,271.821472,90.400002,0.425922,2.299433,...,0.0,309.000000,97.250,5309.041992,231.499786,46.995655,-65.048464,2017-01-05 04:00:00,0.0,500.0
205652,2017-01-05 02:00:00,1.383248e+06,2.438718e+06,272.786316,273.794342,0.00345,271.633972,90.699997,0.238422,2.299433,...,0.0,306.799988,97.250,5307.541992,231.437286,47.004860,-65.085166,2017-01-05 04:00:00,0.0,500.0
205651,2017-01-05 02:00:00,1.383248e+06,2.435715e+06,272.598816,273.419342,0.00338,271.383972,89.699997,0.050922,1.861933,...,0.0,306.600006,97.250,5306.041992,231.437286,47.014054,-65.121879,2017-01-05 04:00:00,0.0,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2017-01-05 23:00:00,1.952773e+04,1.108106e+06,267.236389,267.720215,0.00225,266.037811,90.900002,2.098679,-0.643744,...,0.0,265.200012,84.375,5417.981934,248.687286,37.973587,-84.837865,2017-01-06 01:00:00,0.0,500.0
3,2017-01-05 23:00:00,1.952773e+04,1.105102e+06,266.861389,267.782715,0.00221,265.725311,91.199997,2.223679,-0.581244,...,0.0,262.799988,83.250,5417.513184,248.437286,37.977281,-84.871766,2017-01-06 01:00:00,0.0,500.0
2,2017-01-05 23:00:00,1.952773e+04,1.102098e+06,266.861389,267.720215,0.00221,265.725311,91.099998,2.161179,-0.581244,...,0.0,260.500000,81.875,5417.106934,248.249786,37.980966,-84.905670,2017-01-06 01:00:00,0.0,500.0
1,2017-01-05 23:00:00,1.952773e+04,1.099094e+06,267.173889,267.907715,0.00223,265.912811,90.900002,2.098679,-0.581244,...,0.0,258.200012,80.000,5416.763184,248.124786,37.984641,-84.939578,2017-01-06 01:00:00,0.0,500.0
